In [ ]:
pip install -q tflite-model-maker

     |████████████████████████████████| 616 kB 5.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
     |████████████████████████████████| 237 kB 41.1 MB/s 
     |████████████████████████████████| 840 kB 33.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.3 MB/s 
     |████████████████████████████████| 3.4 MB 33.3 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 1.2 MB 43.4 MB/s 
     |████████████████████████████████| 120 kB 48.6 MB/s 
     |████████████████████████████████| 6.4 MB 26.8 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 99 kB 8.1 MB/s 
     |████████████████████████████████| 352 kB 47.6 MB/s 
     |████████████████████████████████| 47.8 MB 49 kB/s 
     |████████████████████████████████| 462 kB 32.1 MB/s 
     |████████████████

In [ ]:
# Import the required packages

import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

In [ ]:
import pathlib
image_path = "/content/drive/MyDrive/GrapeDoc_Dataset/train"
image_path = pathlib.Path(image_path)

In [ ]:
# Use DataLoader class to load data
# As for from_folder() method, it could load data from the folder
# JPEG-encoded images and PNG-encoded images are supported.
data = DataLoader.from_folder(image_path)

INFO:tensorflow:Load image with size: 7912, num_label: 5, labels: Disease:Black_Measles, Disease:Black_rot, Disease:Leaf_blight, Healthy_leaf, Invalid_Input.


In [ ]:
# Split it to training data (80%), validation data (10%, optional) and testing data (10%).
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [ ]:
# Show 25 image examples with labels.
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Create a custom image classifier model based on the loaded data
model = image_classifier.create(train_data, validation_data=validation_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


197/197 [==============================] - 764s 4s/step - loss: 0.5863 - accuracy: 0.9259 - val_loss: 0.4820 - val_accuracy: 0.9874
Epoch 2/5
197/197 [==============================] - 44s 224ms/step - loss: 0.4929 - accuracy: 0.9764 - val_loss: 0.4636 - val_accuracy: 0.9924
Epoch 3/5
197/197 [==============================] - 44s 223ms/step - loss: 0.4806 - accuracy: 0.9818 - val_loss: 0.4739 - val_accuracy: 0.9924
Epoch 4/5
197/197 [==============================] - 44s 224ms/step - loss: 0.4736 - accuracy: 0.9854 - val_loss: 0.4554 - val_accuracy: 0.9924
Epoch 5/5
197/197 [==============================] - 44s 223ms/step - loss: 0.4683 - accuracy: 0.9852 - val_loss: 0.4476 - val_accuracy: 0.9949


In [ ]:
# Detailed model structure
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________


In [ ]:
# Evaluate the result of the model, get the loss and accuracy of the model.
loss, accuracy = model.evaluate(test_data)

25/25 [==============================] - 101s 4s/step - loss: 0.4522 - accuracy: 0.9886


In [ ]:
# We could plot the predicted results in 100 test images. Predicted labels with red color are the wrong predicted results while others are correct.

# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(50, 50))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Convert the trained model to TensorFlow Lite model format with metadata
model.export(export_dir='/content/drive/MyDrive/Test13')

INFO:tensorflow:Assets written to: /tmp/tmpj_pp_ig8/assets


INFO:tensorflow:Assets written to: /tmp/tmpj_pp_ig8/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp45n19_hx/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp45n19_hx/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/Test12/model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: /content/drive/MyDrive/Test12/model.tflite


In [ ]:
# exporting only the label file.
model.export(export_dir='/content/drive/MyDrive/Test13', export_format=ExportFormat.LABEL)

INFO:tensorflow:Saving labels in /content/drive/MyDrive/Test13/labels.txt


INFO:tensorflow:Saving labels in /content/drive/MyDrive/Test13/labels.txt


In [ ]:
# evaluate the tflite model with the evaluate_tflite method.
model.evaluate_tflite('/content/drive/MyDrive/Test12/model.tflite', test_data)

{'accuracy': 0.98989898989899}